In [1]:
import concurrent.futures
import threading
import multiprocessing

### Задание 6.1


In [2]:
def calc_square(number, result):
    """
    Функция для вычисления квадрата числа и сохранения его в общий результат.
    """
    result.put(number * number)

In [3]:
if __name__ == "__main__":
    # Создаем объект Queue для передачи результатов между процессами
    result_queue = multiprocessing.Queue()

    # Создаем список чисел
    numbers = [1, 2, 3, 4, 5]

    # Создаем список для хранения процессов
    processes = []

    # Создаем процессы для вычисления квадратов чисел
    for number in numbers:
        process = multiprocessing.Process(target=calc_square, args=(number, result_queue))
        processes.append(process)
        process.start()

    # Ждем завершения всех процессов
    for process in processes:
        process.join()

    # Получаем результаты из очереди и выводим их
    while not result_queue.empty():
        print(result_queue.get())

1
4
9
16
25


#### Основные шаги программы:

1. **Определение функции `calc_square`**:
   - Эта функция принимает два аргумента: `number` (число, для которого нужно вычислить квадрат) и `result` (очередь для сохранения результата). Внутри функции происходит вычисление квадрата числа (`number * number`) и результат помещается в очередь `result`.

2. **Использование многопроцессорности**:
   - В блоке `if __name__ == "__main__":` создается объект очереди (`result_queue`) для передачи результатов между процессами.
   - Затем создается список чисел `numbers`, для которых нужно вычислить квадрат.
   - Создается пустой список `processes` для хранения объектов процессов.
   - Для каждого числа из списка `numbers` создается процесс, вызывая функцию `calc_square` с этим числом и объектом `Queue`. Созданные процессы добавляются в список `processes` и запускаются.

3. **Ожидание завершения процессов и вывод результатов**:
   - После запуска всех процессов выполняется цикл `for`, в котором вызывается метод `join()` для каждого процесса. Это блокирует выполнение программы до тех пор, пока соответствующий процесс не завершится.
   - Затем в цикле `while` извлекаются результаты из очереди `result_queue` и выводятся на экран с помощью функции `print()`.

In [4]:
def calculate_square_sum(start, end, result):
    """
    Функция для вычисления суммы квадратов чисел в заданном диапазоне и сохранения результата.
    """
    square_sum = sum(x * x for x in range(start, end + 1))
    result.append(square_sum)

In [5]:
if __name__ == "__main__":
    # Задаем параметры для вычислений
    N = 100000
    num_threads = 4

    # Вычисляем интервал для каждого потока
    interval = N // num_threads

    # Создаем список для хранения результатов
    results = []

    # Создаем список для хранения потоков
    threads = []

    # Создаем и запускаем потоки
    for i in range(num_threads):
        start = i * interval + 1
        end = (i + 1) * interval if i < num_threads - 1 else N
        thread = threading.Thread(target=calculate_square_sum, args=(start, end, results))
        threads.append(thread)
        thread.start()

    # Ждем завершения всех потоков
    for thread in threads:
        thread.join()

    # Вычисляем общую сумму квадратов
    total_square_sum = sum(results)
    print("Общая сумма квадратов чисел от 1 до", N, ":", total_square_sum)

Общая сумма квадратов чисел от 1 до 100000 : 333338333350000


#### Основные шаги программы:

1. **Разделение диапазона на потоки и создание потоков**:
   - Задаются параметры для вычислений: `N` - число, до которого нужно вычислить сумму квадратов, и `num_threads` - количество потоков, которые будут использоваться для вычислений.
   - Вычисляется интервал для каждого потока путем деления общего диапазона `N` на количество потоков `num_threads`.
   - Создается пустой список `results` для хранения результатов вычислений.
   - Создается пустой список `threads` для хранения объектов потоков.
   - В цикле создаются и запускаются потоки для вычисления суммы квадратов в своем интервале. Каждый поток получает свой диапазон значений и ссылку на список `results` для сохранения результата.

2. **Ожидание завершения потоков и вычисление общей суммы**:
   - После запуска всех потоков выполняется цикл, в котором вызывается метод `join()` для каждого потока. Это блокирует выполнение программы до тех пор, пока все потоки не завершат свою работу.
   - После завершения всех потоков суммируются результаты, содержащиеся в списке `results`, для получения общей суммы квадратов чисел.

3. **Вывод общей суммы**:
   - Полученная общая сумма квадратов чисел выводится на экран с помощью функции `print()`.

### Задание 6.2

In [6]:
def multiply_row(row, matrix2):
    """
    Функция для умножения строки матрицы на другую матрицу.
    """
    result_row = []
    for j in range(len(matrix2[0])):
        result_element = sum(row[k] * matrix2[k][j] for k in range(len(row)))
        result_row.append(result_element)
    return result_row

def multiply_matrices(matrix1, matrix2):
    """
    Функция для умножения двух матриц.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Умножаем каждую строку матрицы matrix1 на matrix2 параллельно
        results = [executor.submit(multiply_row, row, matrix2) for row in matrix1]

        # Собираем результаты из Future объектов
        multiplied_matrix = [result.result() for result in concurrent.futures.as_completed(results)]

    return multiplied_matrix

In [7]:
if __name__ == "__main__":
    # Примеры матриц
    matrix1 = [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]

    matrix2 = [
        [9, 8, 7],
        [6, 5, 4],
        [3, 2, 1]
    ]

    result_matrix = multiply_matrices(matrix1, matrix2)
    print("Результат умножения матриц:")
    for row in result_matrix:
        print(row)

Результат умножения матриц:
[84, 69, 54]
[30, 24, 18]
[138, 114, 90]


#### Основные шаги программы:

1. Определение функции `multiply_row(row, matrix2)`, которая принимает строку первой матрицы и вторую матрицу. Внутри функции происходит вычисление элементов результирующей строки, где каждый элемент получается путем суммирования произведений соответствующих элементов строки первой матрицы и столбца второй матрицы.

2. Определение функции `multiply_matrices(matrix1, matrix2)`, которая принимает две матрицы. Внутри этой функции создается `ThreadPoolExecutor`, который автоматически управляет потоками. Каждая строка первой матрицы умножается на вторую матрицу параллельно, и результаты сохраняются в списке `results`. После завершения всех вычислений результаты собираются из объектов `Future`, возвращаемых `submit`, и формируется результирующая матрица `multiplied_matrix`.

3. В блоке `if __name__ == "__main__":` задаются примеры матриц `matrix1` и `matrix2`, и вызывается функция `multiply_matrices` для их умножения. Результаты выводятся на экран.